In [56]:
def extract_topic_subtopic(url):
    try:
        # Remove protocol (https://) and split by slashes
        parts = url.split("//", 1)[-1].split("/")
        
        topic = parts[1] if len(parts) > 1 else "unknown"
        subtopic = parts[2] if len(parts) > 2 and not parts[2].isdigit() else "none"  # if 2nd part exists and is NOT an article ID
    except Exception as e:
        print(f"Error parsing URL {url}: {e}")
        topic = "unknown"
        subtopic = "none"
    
    return topic, subtopic

In [ ]:
def enrich_articles_with_topics(df):
    topics = []
    subtopics = []

    for url in df['url']:
        topic, subtopic = extract_topic_subtopic(url)
        topics.append(topic)
        subtopics.append(subtopic)

    df['topic'] = topics
    df['subtopic'] = subtopics
    return df


In [58]:
url = "https://www.rtvslo.si/sport/preostali-sporti/anka-pogacnik-pot-v-dohi-koncala-v-drugem-krogu/667810"
topic, subtopic = extract_topic_subtopic(url)
print(topic)     # → "slovenija"
print(subtopic)  # → "razsodisce-nekdanja-novinarka-in-urednica-tv-slovenija-krsila-novinarski-kodeks"


sport
preostali-sporti


In [69]:
# --- 1. Imports ---
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
from tqdm import tqdm

# --- 2. Utility functions ---
def load(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    date_feats = np.concatenate([years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]], axis=1)
    return date_feats

def extract_topics_from_url(url):
    parts = url.split('/')
    topic = parts[3] if len(parts) > 3 else 'none'
    subtopic = parts[4] if len(parts) > 4 else 'none'
    return topic, subtopic

# --- 3. Dataset and Model ---
class NewsDataset(Dataset):
    def __init__(self, bert_vectors, topic_ids, subtopic_ids, date_feats, targets):
        self.X = torch.tensor(bert_vectors, dtype=torch.float32)
        self.topic_ids = torch.tensor(topic_ids, dtype=torch.long)
        self.subtopic_ids = torch.tensor(subtopic_ids, dtype=torch.long)
        self.date_feats = torch.tensor(date_feats, dtype=torch.float32)
        self.y = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.topic_ids[idx], self.subtopic_ids[idx], self.date_feats[idx], self.y[idx]

class MLPWithEmbeddings(nn.Module):
    def __init__(self, input_dim, num_topics, num_subtopics):
        super().__init__()
        self.topic_embedding = nn.Embedding(num_topics, 16)
        self.subtopic_embedding = nn.Embedding(num_subtopics, 24)

        self.model = nn.Sequential(
            nn.Linear(input_dim + 16 + 24 + 6, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, 1),
            nn.Softplus()
        )

    def forward(self, x_text, topic_ids, subtopic_ids, date_feats):
        topic_embed = self.topic_embedding(topic_ids)
        subtopic_embed = self.subtopic_embedding(subtopic_ids)
        x = torch.cat([x_text, topic_embed, subtopic_embed, date_feats], dim=1)
        return self.model(x)

# --- 4. Main Model Trainer ---
class RTVSloBERT:
    def __init__(self, batch_size=64, epochs=10, learning_rate=1e-3, l2_lambda=1e-4, eval_split=0.2):
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.eval_split = eval_split

    def fit(self, train_data):
        for a in train_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        raw_targets = [a['n_comments'] for a in train_data]
        targets = [np.log1p(t) for t in raw_targets]
        bert_vectors = torch.load("sloberta_embeddings.pt", map_location="cpu").numpy()

        topics = []
        subtopics = []
        for a in train_data:
            topic, subtopic = extract_topics_from_url(a['url'])
            topics.append(topic)
            subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")

        date_feats = process_date_features(train_data)

        self.topic_encoder = LabelEncoder().fit(topics)
        self.subtopic_encoder = LabelEncoder().fit(subtopics)

        topic_ids = self.topic_encoder.transform(topics)
        subtopic_ids = self.subtopic_encoder.transform(subtopics)

        X_train, X_val, topic_train, topic_val, subtopic_train, subtopic_val, date_train, date_val, y_train, y_val = train_test_split(
            bert_vectors, topic_ids, subtopic_ids, date_feats, targets, test_size=self.eval_split, random_state=42
        )

        train_dataset = NewsDataset(X_train, topic_train, subtopic_train, date_train, y_train)
        val_dataset = NewsDataset(X_val, topic_val, subtopic_val, date_val, y_val)

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        input_dim = bert_vectors.shape[1]
        self.model = MLPWithEmbeddings(input_dim, len(self.topic_encoder.classes_), len(self.subtopic_encoder.classes_))

        self._train(self.model, train_loader, val_loader)

    def _train(self, model, train_loader, val_loader):
        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate, weight_decay=self.l2_lambda)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        criterion = nn.L1Loss()

        best_val_mae = float('inf')
        best_model_state = None

        for epoch in range(self.epochs):
            model.train()
            train_mae_list = []
            for x_batch, topic_ids, subtopic_ids, date_feats, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats).squeeze()
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()

                y_pred_clamped = torch.clamp(y_pred, -10, 10)
                y_batch_clamped = torch.clamp(y_batch, -10, 10)
                mae = torch.mean(torch.abs(torch.expm1(y_pred_clamped) - torch.expm1(y_batch_clamped))).item()
                train_mae_list.append(mae)

            model.eval()
            val_mae_list = []
            with torch.no_grad():
                for x_batch, topic_ids, subtopic_ids, date_feats, y_batch in val_loader:
                    y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats).squeeze()
                    y_pred_clamped = torch.clamp(y_pred, -10, 10)
                    y_batch_clamped = torch.clamp(y_batch, -10, 10)
                    mae = torch.mean(torch.abs(torch.expm1(y_pred_clamped) - torch.expm1(y_batch_clamped))).item()
                    val_mae_list.append(mae)

            train_mae = np.mean(train_mae_list)
            val_mae = np.mean(val_mae_list)
            scheduler.step(val_mae)

            if val_mae < best_val_mae:
                best_val_mae = val_mae
                best_model_state = model.state_dict()

            print(f"Epoch {epoch+1} - Train MAE: {train_mae:.2f} | Val MAE: {val_mae:.2f}")

        if best_model_state is not None:
            model.load_state_dict(best_model_state)

    def predict(self, test_data):
        for a in test_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        bert_vectors = torch.load("sloberta_embeddings_final.pt", map_location="cpu").numpy()

        topics = []
        subtopics = []
        for a in test_data:
            topic, subtopic = extract_topics_from_url(a['url'])
            topics.append(topic)
            subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")

        date_feats = process_date_features(test_data)

        topic_ids = [self.topic_encoder.classes_[0] if t not in self.topic_encoder.classes_ else t for t in topics]
        subtopic_ids = [self.subtopic_encoder.classes_[0] if s not in self.subtopic_encoder.classes_ else s for s in subtopics]

        topic_ids = self.topic_encoder.transform(topic_ids)
        subtopic_ids = self.subtopic_encoder.transform(subtopic_ids)

        X = torch.tensor(bert_vectors, dtype=torch.float32)
        topic_ids = torch.tensor(topic_ids, dtype=torch.long)
        subtopic_ids = torch.tensor(subtopic_ids, dtype=torch.long)
        date_feats = torch.tensor(date_feats, dtype=torch.float32)

        self.model.eval()
        with torch.no_grad():
            preds = self.model(X, topic_ids, subtopic_ids, date_feats).squeeze().numpy()
            return np.clip(np.expm1(preds), 0, None)

# --- 5. Main ---
if __name__ == '__main__':
    train = load("../data/rtvslo_train.json")
    test = load("../data/rtvslo_test.json")

    m = RTVSloBERT(
        eval_split=0.05,
        batch_size=170,
        epochs=50,
        learning_rate=1e-4,
        l2_lambda=1e-3
    )

    m.fit(train)
    p = m.predict(test)
    np.savetxt("predictions.txt", p, fmt="%.4f")


Epoch 1 - Train MAE: 30.77 | Val MAE: 26.56
Epoch 2 - Train MAE: 27.64 | Val MAE: 25.81
Epoch 3 - Train MAE: 26.93 | Val MAE: 25.44
Epoch 4 - Train MAE: 26.33 | Val MAE: 24.92
Epoch 5 - Train MAE: 26.04 | Val MAE: 24.73
Epoch 6 - Train MAE: 25.54 | Val MAE: 25.43
Epoch 7 - Train MAE: 25.48 | Val MAE: 24.44
Epoch 8 - Train MAE: 24.94 | Val MAE: 24.13
Epoch 9 - Train MAE: 24.85 | Val MAE: 24.34
Epoch 10 - Train MAE: 24.75 | Val MAE: 23.75
Epoch 11 - Train MAE: 24.34 | Val MAE: 23.95
Epoch 12 - Train MAE: 24.34 | Val MAE: 23.77
Epoch 13 - Train MAE: 24.04 | Val MAE: 23.96
Epoch 14 - Train MAE: 24.05 | Val MAE: 23.35
Epoch 15 - Train MAE: 23.85 | Val MAE: 24.17
Epoch 16 - Train MAE: 23.66 | Val MAE: 23.61
Epoch 17 - Train MAE: 23.58 | Val MAE: 24.25
Epoch 18 - Train MAE: 23.55 | Val MAE: 23.40
Epoch 19 - Train MAE: 23.42 | Val MAE: 23.47
Epoch 20 - Train MAE: 23.20 | Val MAE: 23.86
Epoch 21 - Train MAE: 22.73 | Val MAE: 22.98
Epoch 22 - Train MAE: 22.70 | Val MAE: 22.73
Epoch 23 - Train MA

In [67]:
import numpy as np
import json
from sklearn.metrics import mean_absolute_error

# 1. Load your predictions
preds = np.loadtxt("predictions.txt")

# 2. Load your true y-values from dataset_val.json
with open("../data/rtvslo_validation.json", "r", encoding="utf-8") as f:
    val_articles = json.load(f)

# 3. Extract true n_comments
y_true = np.array([a["n_comments"] for a in val_articles], dtype=np.float32)

# 4. Check lengths
assert len(preds) == len(y_true), f"Length mismatch: preds={len(preds)}, y_true={len(y_true)}"

# 5. Calculate MAE
mae = mean_absolute_error(y_true, preds)

print(f"📊 MAE between predictions and true values: {mae:.2f}")


📊 MAE between predictions and true values: 27.13


## ------------------

In [ ]:
# --- 1. Load best model ---
model.load_state_dict(torch.load("best_model_topic.pt"))  # ⚡ Match training model name!
model.eval()

# --- 2. Prepare validation data ---
val_df = pd.read_json("../data/rtvslo_validation.json")
val_df = enrich_articles_with_time_features(val_df)

# Load sloberta validation embeddings
text_embeddings_val = torch.load("sloberta_embeddings_val.pt", weights_only=True)
targets_val = torch.load("targets_val.pt", weights_only=True)

# Process time features
time_features_val, _ = process_time_features(val_df, scaler=time_scaler)

# Encode topics/subtopics using the SAME encoders!
topic_ids_val = torch.tensor(topic_encoder.transform(val_df['topic']), dtype=torch.long)
subtopic_ids_val = torch.tensor(subtopic_encoder.transform(val_df['subtopic']), dtype=torch.long)

# --- 3. TensorDataset and DataLoader ---
val_dataset = TensorDataset(text_embeddings_val, time_features_val, topic_ids_val, subtopic_ids_val, targets_val)
val_loader = DataLoader(val_dataset, batch_size=128)

# --- 4. Evaluation loop ---
y_preds = []
y_trues = []

with torch.no_grad():
    for text, time, topic_id, subtopic_id, y in val_loader:
        text, time, topic_id, subtopic_id, y = text.to(device), time.to(device), topic_id.to(device), subtopic_id.to(device), y.to(device)
        y_pred = model(text, time, topic_id, subtopic_id)
        y_preds.append(torch.expm1(y_pred).cpu())  # 🔥 Reverse log1p
        y_trues.append(torch.expm1(y).cpu())

y_preds = torch.cat(y_preds).numpy()
y_trues = torch.cat(y_trues).numpy()

# --- 5. Metrics ---
mae = mean_absolute_error(y_trues, y_preds)
rmse = np.sqrt(mean_squared_error(y_trues, y_preds))
r2 = r2_score(y_trues, y_preds)

print("\n📊 Validation Results with Topics:")
print(f"  MAE : {mae:.2f}")
print(f"  RMSE: {rmse:.2f}")
print(f"  R²  : {r2:.4f}")


ValueError: y contains previously unseen labels: 'dopolnitev-strategije-drzavnih-nalozb-cilj-invalidskih-podjetij-tudi-druzbeno-odgovorno-upravljanje'

### Test set output

In [48]:
import torch
import pandas as pd
import numpy as np

# 1. Load your saved model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LinearRegressionPredictor(
    input_dim_text=text_embeddings.shape[1],  # match your trained model
    input_dim_time=6,                         # if you use only days_since
    hidden_dim=128,
    dropout_prob=0.5
).to(device)
model.load_state_dict(torch.load("best_model_linear_log.pt", map_location=device))
model.eval()

# 2. Load new articles
df = pd.read_json("../data/rtvslo_test.json")  # or whatever your test set is

# 3. Preprocess time features
df['date'] = pd.to_datetime(df['date'])
    
# Extract raw features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.weekday  # Monday = 0
df['hour'] = df['date'].dt.hour

df['year_scaled'] = year_scaler.transform(df['year'].values.reshape(-1, 1))
df['month_scaled'] = month_scaler.transform(df['month'].values.reshape(-1, 1))

df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Only using 'days_since' here
time_features_new = torch.tensor(
    df[['year_scaled', 'month_scaled', 'day_sin', 'day_cos', 'hour_sin', 'hour_cos']].values,
    dtype=torch.float32
)

# 4. Load the corresponding SloBERTa embeddings for new articles
text_embeddings_new = torch.load("sloberta_embeddings_final.pt", weights_only=True)

# 5. Predict
model.eval()
preds = []

batch_size = 128
dataset = torch.utils.data.TensorDataset(text_embeddings_new, time_features_new)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
print(len(dataset))

with torch.no_grad():
    for text, time in loader:
        text, time = text.to(device), time.to(device)
        y_pred = model(text, time)
        y_pred = torch.expm1(y_pred)  # 🔥 reverse log1p to real counts
        preds.append(y_pred.cpu())

# Stack predictions
preds = torch.cat(preds).numpy()
print(len(preds))

# 6. Save predictions to .txt
np.savetxt("final_predictions.txt", preds, fmt="%.3f")  # or "%.0f" if you want integer predictions

print("✅ Saved predictions to final_predictions.txt")

2218
2218
✅ Saved predictions to final_predictions.txt
